# Introduction

Welcome to the **Capstone Project focused on Biological Microscopy Images**!

***Before starting this notebook, make a local copy in your Google Drive by selecting File > Save a Copy in Drive.***

>This notebook is designed to be modular. Each time you return to this notebook, first run the "Introduction" section to install and import packages, and load data and metadata. Then, you can run each subsequent section independently (including "Exploratory Data Analysis", "Radiation Type Classifiers", "Sex Classifiers", "Strain Classifiers").

----

In this notebook, just like in the [Bioimaging Notebook](https://colab.research.google.com/drive/1n3PyMRgfrb4fWD53fgSuhtV364qUaW4r?usp=sharing), we will be using data from the [NASA Biological and Physical Sciences Microscopy Benchmark Training dataset](https://registry.opendata.aws/bps_microscopy/), publicly available on the Amazon Web Services Open Data Registry.

Recall that this dataset contains thousands of images of cellular nuclei from mouse fibroblast cells that have been irradiated with X-rays or high-energy iron (Fe) particles at different doses, and imaged at different timepoints.

The cell nuclei were then stained with a fluorescent marker that adheres to areas of DNA damage and repair, called *53bp1*.

Thus, when viewing the images, we can see that the brightly fluorescent spots in the cell nucleus indicate DNA damage due to radiation exposure. This allows us to study the impact of radiation exposure on the DNA within cells. These bright spots are called DNA damage *foci*.

<img src="https://drive.google.com/uc?id=1OS3kJDg88GPwZeFcCEKtr3eDANj4nBw_">


More information on this dataset is available [here](https://docs.google.com/document/d/e/2PACX-1vTIjUPenLxVX0stErsBbK884QMJW_Ur1mqHJ9K3KIZl3klT90cxHDppsEvz5Z6Skdu13X8tzghqyWcN/pub).

----

In this Capstone Research Project, you will be using machine learning methods to analyze the dataset. Here are some of the questions you will be answering:

- What patterns are present when we cluster the dataset, and what does that tell us about the biological similarities between images in the dataset?
- Can we train a machine learning classifier to classify radiation type (X-ray vs. Fe)?
  - What type of classifier works best for this task? Why?
- What do the misclassified images have in common? What about the correctly classified images?
  - What does this tell us about what biological patterns are being used by the models?
- Can we train a machine learning classifier to classify the *sex* of the mouse from which each cell nucleus image was taken?
  - What about the *strain* of the mouse?
  - If not, what does this tell us about what biological traits are being captured in the radiation images?


After working through these questions, you will have the chance to independently pursue additional questions if you have the time and interest. These questions are completely optional, and you can also return to them after this course is finished:

- Does a pre-trained image classifier perform better than a from-scratch model?
- Can you train a regression algorithm to identify the number of DNA damage *foci* in an image?
- Can you design a method to count the number of DNA damage *foci* in an image?


## Installing and Importing Packages

First, let's install and import packages needed for reading in the data and metadata.

The `s3fs` library in Python is used to mount an Amazon S3 bucket so that you can operate in files and directories in an S3 bucket like a local file system. Since our dataset is stored on the Amazon Web Services Open Data Registry, the `s3fs` library will allow us to access the dataset without downloading it.

The `boto3` library is another Python library for interacting with Amazon S3 services.

In [ ]:
# Install s3fs library
!pip install s3fs
# Install boto3 library
!pip install boto3
# Import packages
from botocore import UNSIGNED
from botocore.config import Config
import boto3
import pandas as pd
import os


In [ ]:
# Define a function to load image from S3 using the index
def load_image_from_s3(index, labels_df):
  row = labels_df.iloc[index]
  img_filename = row['filename']
  obj = s3_client.get_object(Bucket=s3_bucket, Key=os.path.join(s3_dir_path, img_filename))
  img_bytes = obj['Body'].read()
  img = Image.open(BytesIO(img_bytes))
  return img

## Read In Metadata

In order to train a machine learning model on this image dataset, we need to know as much as possible about each image.

Fortunately, lots of metadata has been published about each image. Let's read it in!

### Metadata About the Image Files

First, we'll mount the Amazon S3 bucket that contains the image data and its associated metadata table, which is called `meta.csv`. We'll use the `pandas` library to read in the metadata table, perform some filtering, and take a look at the final table.


> *Filtering Steps:*
In order to reduce compute times, we will filter the metadata to only include images that were taken 4 hours after radiation exposure (when the DNA damage signal is strongest), and we will select a random 10% subset of those images.

In [ ]:
# Mount the S3 bucket
#s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))
s3_client = boto3.client('s3', config=Config(signature_version=UNSIGNED))

# Define the bucket and directory within the bucket to our data
s3_bucket = 'nasa-bps-training-data'
s3_dir_path = "Microscopy/train"

# Define the directory path as a string and convert to os.PathLike object
image_dir = os.fspath(f"s3://{s3_bucket}/{s3_dir_path}")

In [ ]:
# Load labels from CSV
meta = pd.read_csv(image_dir+'/meta.csv')

# Get only the 4hr timepoint images for the radiation classifiers
meta_4hr = meta[(meta['hr_post_exposure'] == 4)]

# Sample 10% from each class to limit compute needs
sub_meta_4hr = meta_4hr.groupby('particle_type').apply(lambda x: x.sample(frac=0.1, random_state=42))
sub_meta_4hr = sub_meta_4hr.reset_index(drop=True) # Reset index to avoid multi-index

meta = sub_meta_4hr

# Pull out plate and well from filename into their own column to use for mapping later
meta['plate_well'] = [x.split('_')[0] + '_' + x.split('-')[1].split('_')[0] for x in meta['filename']]

meta

Take a look at the the `meta.csv` table, which has 5 different columns with information about each image.

**CHALLENGE QUESTIONS:**
Take a look at the [documentation](https://docs.google.com/document/d/e/2PACX-1vTIjUPenLxVX0stErsBbK884QMJW_Ur1mqHJ9K3KIZl3klT90cxHDppsEvz5Z6Skdu13X8tzghqyWcN/pub) for this dataset. What do each of the columns tell us?






**Double click here to enter your answers to the questions in this text box.**
<font color="green" size=5>
Answers:
<br>
<ol>
<li>
"filename"
</li>
<br>
<li>
"dose_Gy"
</li>
<br>
<li>
"particle_type"
</li>
<br>
<li>
"hr_post_exposure"
</li>
<br>
<li>
"plate_well"
</li>
</ol>
</font>

### Metadata About the Mice

Now, let's take a look at some metadata that has been published about the mice that were used in this study. The information about the mice is available through the [NASA Open Science Data Repository (OSDR)](https://osdr.nasa.gov/bio/), in study [OSD-366](https://osdr.nasa.gov/bio/repo/data/studies/OSD-366).

First, let's read in all the information we have about the mice using the `pandas` library and the [OSDR Public API](https://genelab.nasa.gov/genelabAPIs) which allows us to use Python code to access the contents of OSDR studies.

In [ ]:
# Read in the sample metadata from OSD-366 using the OSDR Public API
url = 'https://osdr.nasa.gov/geode-py/ws/studies/OSD-366/download?source=datamanager&file=GLDS-366_Histology_raw_pheno_V3.csv'

sampleMeta = pd.read_csv(url)[['Sample Name', 'plate', 'plate_well', 'Strain', 'Gender', 'avg_foci_no_outl']]
sampleMeta = sampleMeta.rename(columns={'plate_well': 'well', 'Gender': 'Sex'})
sampleMeta['plate_well'] = sampleMeta['plate'] + '_' + sampleMeta['well']
sampleMeta.drop(['Sample Name', 'plate', 'well'], axis=1, inplace=True)
sampleMeta

Take a look at the table above. The different columns in this table tell us lots of information about the mice:
- "plate_well" = the plate and well number (remember each well in a 96-well plate contains cells from one mouse)
- "Strain" = the strain of the mouse
- "Sex" = the sex of the mouse
- "avg_foci_no_outl" = the number of DNA damage foci found in that well

**CHALLENGE QUESTION:** If we want to combine the metadata about the mice with the metadata about the images, is there a column that both of these tables have in common that we could use to map between the tables? If so, what is it?

**Double click here to enter your answers to the questions in this text box.**

<font color="green" size=5>
Answer:
<br>
</font>

----
Now, let's combine the metadata about the images with the metadata about the mice, so that we have a single metadata table with all the information we need to train machine learning models.

We will use Python *dictionaries* for this task in the code blocks below. We will use the "plate-well" column in each table to map the two tables together.

In [ ]:
# Create dictionaries that relate plate-well and sex or strain
sexDict = sampleMeta[['plate_well', 'Sex']].set_index('plate_well').to_dict()['Sex']
strainDict = sampleMeta[['plate_well', 'Strain']].set_index('plate_well').to_dict()['Strain']
fociDict = sampleMeta[['plate_well', 'avg_foci_no_outl']].set_index('plate_well').to_dict()['avg_foci_no_outl']

In [ ]:
# Collect sex and strain and nfoci for all images, using the plate-well combinations
# Add to metadata table

sexMap = []
for x in meta['plate_well']:
  sexMap.append(sexDict[x])
meta['sex'] = sexMap

strainMap = []
for x in meta['plate_well']:
  strainMap.append(strainDict[x])
meta['strain'] = strainMap

fociMap = []
for x in meta['plate_well']:
  fociMap.append(fociDict[x])
meta['avg_foci_no_outl'] = fociMap

meta

> Nice work! We now have a single metadata table with the filename of each image, plus several different **classes** for each image to train our classifiers.

## Read in the Image Data

Now, we need to load the image data into memory and preprocess it in preparation for training machine learning algorithms.

First, we'll import a few packages that are useful for image preprocessing such as `matplotlib` and `PIL`.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
# from sklearn.decomposition import PCA
# from sklearn.manifold import TSNE
# from sklearn.preprocessing import StandardScaler
from io import BytesIO

Next, we'll write a Python function which loads all the images and performs some preprocessing. This function:
- Loads each image into memory
- Filters out any images with pixel intensity greater than 4000. The settings on the microscope that captured these images should only allow pixel intensities up to 4000. *Any images with pixel intensities greater than 4000 indicate an error during the automated image capture process, so we filter out those low quality images.*
- Calculates the dimensions (width, height) of each image so that we can analyze the size distribution across the dataset. *This is important because most neural network classifiers require all images to be the same size.*

Then we'll use that function. **NOTE: the next 2 cells take a few minutes to run.**

In [ ]:
# Function to load and get image sizes
def load_images(file_list):
    images = []
    images_resized = []
    bad_images = []
    for file in file_list:
      obj = s3_client.get_object(Bucket=s3_bucket, Key=s3_dir_path+'/'+file)
      img_data = obj['Body'].read()
      img = Image.open(BytesIO(img_data))
      if np.array(img).max() < 4000: # filter out any images with pixel intensity > 4000 (low quality images)
        img_resize = img.resize((128, 128))
        images.append(img) # for evaluating the original sizes of the images
        images_resized.append(img_resize) # for training classifiers
      else:
        bad_images.append(file)

    return images, np.array(images_resized), bad_images

In [ ]:
# Load and preprocess images - this takes a few minutes
image_files = meta['filename'].tolist()
loaded_images, resized_images, filtered_images = load_images(image_files)

In [ ]:
len(filtered_images)

The code above prints out the length of the `filtered_images` variable, which holds a list of all the images that were filtered out due to pixel intensity greater than 4000.

**CHALLENGE QUESTION:** How many poor quality images were filtered out of the dataset on the basis of high pixel intensity?

**Double click here to enter your answers to the questions in this text box.**

<font color="green" size=5>
Answer:
</font>

In [ ]:
# Remove filtered images from metadata table
meta = meta[~meta['filename'].isin(filtered_images)]
# Reset the index for meta to be between 0 and 2438
meta = meta.reset_index(drop=True)
meta

The final metadata table is shown above, after filtering out low quality images.

Take a look at the table dimensions, shown at the bottom left.

**CHALLENGE QUESTION:** How many images are we left with for training our classifiers?

**Double click here to enter your answers to the questions in this text box.**
<font color="green" size=5>
Answer:
</font>

### Get Image Paths

Finally, let's collect the paths to each image that we will use for training our classifiers. Although we already loaded in the data, this will be handy for accessing individual images later.

In [ ]:
# Get the filenames of all images in the bucket
s3_client = boto3.client('s3', config=Config(signature_version=UNSIGNED))  # Creating a client for accessing S3 services

paginator = s3_client.get_paginator('list_objects_v2')
pages = paginator.paginate(Bucket=s3_bucket, Prefix=s3_dir_path)

all_paths = []
for page in pages:
    for obj in page['Contents']:
        all_paths.append(obj['Key'])

len(all_paths)

In [ ]:
# Get paths to only the images in our subset metadata table

paths_for_training = []
for x in all_paths:
  if x.split('/')[2] in list(meta['filename']):
    paths_for_training.append(x)

len(paths_for_training)

----

# Exploratory Data Analysis (EDA)

Exploratory Data Analysis (EDA) is a fundamental first step in using machine learning algorithms to analyze biological datasets. Before training any algorithm, we need to understand the structure of the data and any patterns, structures, or anomalies that would affect our analysis.

In our case, we will be performing an EDA to understand how the different classes in our dataset (radiation exposure, dose, sex, strain) relate to each other. We will also be looking for outliers in the dataset, and for any imbalance in the classes (e.g. significantly more samples of one class than another).

We will use techniques such as Principal Component Analysis (PCA), bar graphs, histograms, and image visualization techniques that we learned in the [Bioimaging Notebook](https://colab.research.google.com/drive/1n3PyMRgfrb4fWD53fgSuhtV364qUaW4r?usp=sharing).

## Dimensionality Reduction Using PCA

First, let's perform dimensionality reduction using PCA and then visualize the top two dimensions in our dataset using a scatter plot. This will allow us to visualize the top sources of variation in our dataset and look for patterns.

We can create multiple PCA plots, colored by the different classes, to look for patterns within each class.

This code has multiple steps:

**1. Flatten the Images:**

Typically, images are represented within code as 2D arrays (matrices) of pixel values. For example, a grayscale image of size 5x5 pixels is represented as a 5x5 matrix.
<img src="https://drive.google.com/uc?id=1ylB588B0eYLGbHXXmgzFTKCdEjR8AL7x">

Flattening the images means transforming these multi-dimensional arrays into 1D vectors. For instance, a 5x5 grayscale image will be converted into a vector of length 25 (since 5*5=25).

Algorithms like PCA typically expect the input data to be in a 2D format, where each row represents an observation (in this case, an image) and each column represents a feature (in this case, a pixel value).


**2. Perform Dimensionality Reduction using t-SNE:**

We then perform dimensionality reduction across the dataset using the [t-SNE algorithm](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html), which is similar to PCA. You can revisit PCA in the [Introduction to Clustering Lecture](https://www.youtube.com/watch?v=XPJmokNE7jY) and the [Clustering Notebook](https://colab.research.google.com/drive/1DdgTQUQ0hbNQxOfxK28kbA-z4Ilnof18?usp=sharing).

t-SNE (t-Distributed Stochastic Neighbor Embedding) is a non-linear dimensionality reduction technique which we are using to visualize high-dimensional image data in 2 dimensions.

First, we fit the t-SNE model to the data, which learns the internal structure. Then, we transform the data into the 2-dimensional space based on the learned structure. The output is a 2D array where each row corresponds to the same image in the input data but represented in the 2D space learned by t-SNE.

**3. Plot the First Two Dimensions:**

Finally, we plot the reduced dimensionality data in a scatter plot and color each point by the different groups within classes, to look for patterns.

In [ ]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import random

In [ ]:
# Flatten the images
X = resized_images.reshape(resized_images.shape[0], -1)

# Perform dimensionality reduction using t-SNE
tsne = TSNE(n_components=2, random_state=42)
X_tsne = tsne.fit_transform(X)

# Plot clusters color-coded by different labels
plt.figure(figsize=(12, 12))

# Plot and color by particle type
plt.subplot(2, 2, 1)
for label in meta['particle_type'].unique():
    indices = meta[meta['particle_type'] == label].index # capture indices of images belonging to each class
    plt.scatter(X_tsne[indices, 0], X_tsne[indices, 1], label=label, alpha=0.5) # match each index to its corresponding row in the reduced dim TSNE
plt.title('Particle Type')
plt.legend()

# Plot and color by dose
plt.subplot(2, 2, 2)
for label in meta['dose_Gy'].unique():
    indices = meta[meta['dose_Gy'] == label].index
    plt.scatter(X_tsne[indices, 0], X_tsne[indices, 1], label=label, alpha=0.5)
plt.title('Dose (Gy)')
plt.legend()


# Plot and color by sex
plt.subplot(2, 2, 3)
for label in meta['sex'].unique():
    indices = meta[meta['sex'] == label].index
    plt.scatter(X_tsne[indices, 0], X_tsne[indices, 1], label=label, alpha=0.5)
plt.title('Sex')
plt.legend()

# Plot and color by sex
plt.subplot(2, 2, 4)
for label in meta['strain'].unique():
    indices = meta[meta['strain'] == label].index
    plt.scatter(X_tsne[indices, 0], X_tsne[indices, 1], label=label, alpha=0.5)
plt.title('Strain')
plt.legend()

plt.show()


**CHALLENGE QUESTIONS:**

1. Take a look at the PCA plots above. Which class separates the dataset into clusters the best?
2. Which class(es) do NOT separate the dataset, where the different groups are fully mixed throughout the dataset?
3. For the class that separates the dataset the best, is it a full separation? Or is there some overlap between classes? If there is overlap, where in the cluster plot are the samples overlapping the most? What does this tell you biologically about the data?

**Double click here to enter your answers to the questions in this text box.**

<font color="green" size=5>
Answers:
<br>
<ol>
<li>

</li>
<br>
<li>

</li>
<br>
<li>

</li>
</ol>
</font

## Breakdown of Classes

For training a machine learning algorithm, especially a binary classifier, it is important that the classes be approximately balanced (50% of the data in each class).

Let's create some bar graphs to examine how balanced the dataset is in terms of radiation particle type exposure, mouse sex, and mouse strain.

In [ ]:
# Group by particle_type and sex, and count the occurrences
grouped = meta.groupby(['particle_type', 'sex']).size().unstack()

# Plotting
ax = grouped.plot(kind='bar', stacked=True, figsize=(7, 4))

# Adding titles and labels
plt.title('Distribution of Sex for Each Particle Type')
plt.xlabel('Particle Type')
plt.ylabel('Count')
plt.xticks(rotation=0)

# Place the legend outside the plot
ax.legend(title='Sex', bbox_to_anchor=(1.05, 1), loc='upper left')

# Show plot
plt.tight_layout()
plt.show()


In [ ]:
# Group by particle_type and sex, and count the occurrences
grouped = meta.groupby(['particle_type', 'strain']).size().unstack()

# Plotting
ax = grouped.plot(kind='bar', stacked=True, figsize=(7, 4))

# Adding titles and labels
plt.title('Distribution of Strain for Each Particle Type')
plt.xlabel('Particle Type')
plt.ylabel('Count')
plt.xticks(rotation=0)

# Place the legend outside the plot
ax.legend(title='Strain', bbox_to_anchor=(1.05, 1), loc='upper left')

# Show plot
plt.tight_layout()
plt.show()


**CHALLENGE QUESTIONS:**

Taking a look at the bar plots above, how balanced is the dataset for the following classes?

1. Particle type
2. Sex
3. Strain
4. Do you have any data balance concerns about training a binary classifier on this data, to classify each of these classes?

**Double click here to enter your answers to the questions in this text box.**

<font color="green" size=5>
Answers:
<br>
<ol>
<li>

</li>
<br>
<li>

</li>
<br>
<li>

</li>
<br>
<li>

</li>
</ol>
</font>

## Image Size Distributions

To train a neural network on an imaging dataset, we need to ensure that our images are all the same size. As we discussed in the Bioimaging Notebook, this can be problematic if the images are very different sizes, since it can introduce artifacts into the data.

Let's look at the distributions of image sizes (width and height) across classes to see whether resizing is appropriate. We can plot image width across the dataset as a histogram, to see if most of the images are around the same width for both the Fe and X-ray groups. We can do the same thing for the image height.


First, we need to collect the width and height for all images in both groups.
> **The next cell takes a minute or so to run.**

In [ ]:
# Collect width and height for the Fe and X-ray particle type classes
xray=[]
fe=[]
for im, path in zip(loaded_images, paths_for_training):
  w, h = im.size
  particle = meta.set_index('filename').to_dict()['particle_type'][path.split('/')[2]]
  if particle == 'X-ray':
    xray.append((w,h))
  if particle == 'Fe':
    fe.append((w,h))

Next, let's plot a histogram of ***image width*** across our dataset, split into the X-ray and Fe groups from the particle type class.

In [ ]:
plt.hist([x[0] for x in xray], label='X-ray', alpha=0.5)
plt.hist([x[0] for x in fe], label='Fe', alpha=0.5)
plt.ylabel('Number of images')
plt.xlabel('Image Width (pixels)')
plt.legend()
plt.title('Width')

# Get mean width for X-ray and Fe
print('Mean Width X-ray Images: ' + str(np.mean([x[0] for x in xray])))
print('Mean Width Fe Images: ' + str(np.mean([x[0] for x in fe])))

Next, let's plot a histogram of ***image height*** across our dataset, split into the X-ray and Fe groups from the particle type class.

In [ ]:
plt.hist([x[1] for x in xray], label='X-ray', alpha=0.5)
plt.hist([x[1] for x in fe], label='Fe', alpha=0.5)
plt.ylabel('Number of images')
plt.xlabel('Image Height (pixels)')
plt.legend()
plt.title('Height')

# Get mean height for X-ray and Fe
print('Mean Height X-ray Images: ' + str(np.mean([x[1] for x in xray])))
print('Mean Height Fe Images: ' + str(np.mean([x[1] for x in fe])))

**CHALLENGE QUESTIONS:**

1. Taking a look at the histograms above, do the X-ray and Fe images appear to follow a similar distribution in terms of width? Height?
2. Are the width and height of the images in both groups spread out, or do the majority of the images seem to have a similar width or height?
3. What is the approximate value of the width of the majority of the images? Height? (*Hint:* Take a look at the mean values printed out before each plot.)

**Double click here to enter your answers to the questions in this text box.**

<font color="green" size=5>
Answers:
<br>
<ol>
<li>

</li>
<br>
<li>

</li>
<br>
<li>

</li>
</ol>
</font

----

> **NOTE:** Based on the results of the image size distributions, **we will proceed with resizing our images for neural networks to a consistent square size of 140x140 pixels**. This is slightly larger than the mean for both height and width, since the distribution is slightly skewed toward the larger side (the right side of each plot). Because both of our main groups (X-ray and Fe exposure) have very similar distributions, we do not expect resizing to introduce any artifacts in one group that are not also present in the other, so we will not be impacting the performance of our neural networks.

----

## Selected Image Visualizations

Let's take a look at a few of the images in our dataset.

Recall that the X-ray exposure causes a diffuse or "cloudy" DNA damage signal, which sometimes includes distinct spots of DNA damage without a specific pattern.

Recall also that the Fe heavy ion exposure can leave a characteristic linear "track" of DNA damage through the cell, but if the Fe ion did not directly impact the cell, there may not be any track. In this case, the Fe exposure can appear more like the X-ray exposure.

We'll image 10 random Xray-exposed samples, and 10 random Fe-exposed samples.

>*You can rerun these cells many times to get a different selection of 10 random images.*

----

**10 Random Xray-exposed Images:**

In [ ]:
# Select 10 random X-ray images
random_xray_indices = np.random.choice(meta[meta['particle_type']=='X-ray'].index, 10, replace=False)

# Create a figure for the grid
fig, axes = plt.subplots(2, 5, figsize=(10, 5))
axes = axes.flatten()

# Plot each misclassified image
for ax, idx in zip(axes, random_xray_indices):
  img = load_image_from_s3(idx, meta)
  ax.imshow(img, vmin=400, vmax=4000)

fig.suptitle('10 Random Xray-exposed Images', fontsize=16)

plt.tight_layout()
plt.show()

---
**10 Random Fe-exposed Images:**

In [ ]:
# Select 10 random Fe images
random_Fe_indices = np.random.choice(meta[meta['particle_type']=='Fe'].index, 10, replace=False)

# Create a figure for the grid
fig, axes = plt.subplots(2, 5, figsize=(10, 5))
axes = axes.flatten()

# Plot each misclassified image
for ax, idx in zip(axes, random_Fe_indices):
  img = load_image_from_s3(idx, meta)
  ax.imshow(img, vmin=400, vmax=4000)

fig.suptitle('10 Random Fe-exposed Images', fontsize=16)

plt.tight_layout()
plt.show()


**CHALLENGE QUESTIONS:**

1. Re-run the X-ray cell a few times. What patterns show up consistently? Does this make sense given how X-rays cause DNA damage patterns within cells?

2. Re-run the Fe cell a few times. How often does a cell with a distinct linear track come up? What other patterns do you see?

3. Based on the differences and similarities between X-ray and Fe exposed cells, do you expect it to be easy or difficult for a machine learning classifier to learn the differences between the two types of radiation exposure?

**Double click here to enter your answers to the questions in this text box.**

<font color="green" size=5>
Answers:
<br>
<ol>
<li>

</li>
<br>
<li>

</li>
<br>
<li>

</li>
</ol>
</font>

----

# Radiation Type Classifiers

Now that we've completed our EDA, let's focus on the questions we outlined at the beginning of this Capstone Project.

- **Can we train a machine learning classifier to classify radiation type (X-ray vs. Fe)?**
  - **What type of classifier works best for this task? Why?**
- **What do the misclassified images have in common? What about the correctly classified images?**
  - **What does this tell us about what biological patterns are being used by the models?**

To answer these questions, we'll explore the use of two powerful and widely-used machine learning classifiers: Support Vector Machines (SVM) and Convolutional Neural Networks (CNN). These classifiers have proven effective in various classification tasks, but they have distinct characteristics and strengths that make them suitable for different types of data and problems.

### Support Vector Machines (SVM)

SVMs are a type of supervised learning algorithm that can be used for both classification and regression challenges. The key idea behind SVMs is to find the optimal hyperplane that separates the data points of different classes with the maximum margin. Here are some of the main features of SVMs:

- **High-dimensional space handling:** SVMs perform well in high-dimensional spaces and are effective when the number of dimensions exceeds the number of samples.
- **Effective with small datasets:** SVMs are particularly useful when the dataset is small but well-defined.
- **Kernel trick:** SVMs can handle non-linear data through the use of kernel functions, which transform the data into a higher-dimensional space where a linear separator can be found.

In our context, we will train an SVM classifier to distinguish between X-ray and Fe radiation types using features extracted from the dataset. We'll evaluate the performance of the SVM using metrics such as accuracy, precision, recall, and F1-score. We will also take a look at the support vector plot, and the incorrectly classified images, to understand the SVM model's strengths and weaknesses.

### Convolutional Neural Networks (CNN)

CNNs are a class of deep learning models specifically designed to process and analyze image data. They are highly effective in recognizing patterns and structures in images due to their convolutional layers, which automatically learn spatial hierarchies of features. Here are some key characteristics of CNNs:

- **Automatic feature extraction:** CNNs automatically learn relevant features from raw image data, eliminating the need for manual feature extraction.
- **Translation invariance:** The convolutional layers make CNNs robust to shifts and translations in the input images.
- **Scalability:** CNNs can scale to large datasets and complex tasks, often outperforming traditional machine learning models in image classification tasks.

For our task, we will train a CNN to classify images of radiation types (X-ray vs. Fe). The CNN will learn from raw image data and extract features that help in distinguishing between the two types. We will assess the CNN's performance using the same set of evaluation metrics as for the SVM.

### Comparison and Evaluation

To determine which classifier works best for classifying radiation types, we will compare the performance of the SVM and CNN models based on the following criteria:

- **Accuracy:** The proportion of correctly classified samples out of the total samples.
- **Precision:** The ratio of true positive predictions to the total positive predictions, indicating the accuracy of positive predictions.
- **Recall:** The ratio of true positive predictions to the total actual positives, indicating the ability to identify all positive samples.
- **F1-score:** The harmonic mean of precision and recall, providing a balance between the two metrics.

By training both models and evaluating their performance on a test set, we will gain insights into the strengths and weaknesses of SVMs and CNNs for this specific classification task. This comparison will help us identify the most suitable classifier for distinguishing between X-ray and Fe radiation types and understand the underlying reasons for its effectiveness.



First, let's isolate only the "particle_type" column from the metadata table. This is the column with information on which radiation type the cell in each image was exposed to (Fe or X-ray).

In [ ]:
# Get the particle labels only
particle_labels = meta[['filename','particle_type']]
particle_labels

## SVM

Next, let's train the SVM. We're using the SVC implementation of SVM in scikit-learn (sklearn). You can read more about this implementation [here](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html).

**Steps to train the SVM:**
- Encoding labels: We encode the class labels ("Fe" and "X-ray") as numerical values (e.g. 0 and 1), since the SVM only takes numerical data.
- Split data into 80% training and 20% testing
- Flatten image arrays: flatten each 2D array into a 1D vector for input to the SVM
- Train the SVM model using a Radial Basis Function (RBF) kernel. Because it relies on a Gaussian function, the RBF kernel is often the best for image classification tasks.
- Perform predictions using the held-out 20% training data in order to calculate the accuracy of the model.

In [ ]:
# Import libraries for SVM and downstream analysis
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import seaborn as sns
from sklearn.metrics import confusion_matrix

import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import random

In [ ]:
# Encoding labels
label_encoder = LabelEncoder()
particle_labels['particle_type_encoded'] = label_encoder.fit_transform(particle_labels['particle_type'])

# Set up X and y variables for data and labels
X = resized_images
y = particle_labels['particle_type_encoded'].values

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Flatten image arrays
X_train_flat = X_train.reshape(X_train.shape[0], -1)
X_test_flat = X_test.reshape(X_test.shape[0], -1)

# Train SVM model
svm_model = SVC(kernel='rbf')
svm_model.fit(X_train_flat, y_train)

# Predictions
y_pred = svm_model.predict(X_test_flat)

# Evaluate model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

> Take a look at the classification report from the SVM printed above.

**CHALLENGE QUESTIONS:**

1. What is the overall accuracy of the SVM model?
2. How many images from each class were used in the test data to calculate the accuracy and other metrics? (*Hint:* Take a look at the "support" column.) Are the classes approximately equally split?

**Double click here to enter your answers to the questions in this text box.**

<font color="green" size=5>
Answers:
<br>
<ol>
<li>

</li>
<br>
<li>

</li>
</ol>
</font>

#### Confusion Matrix

Now, let's generate the confusion matrix as another way to visualize the correctly and incorrectly classified images.

As a reminder, a confusion matrix summarizes the performance of a classification model by comparing its predicted class labels against the true class labels, to show how many samples are correctly and incorrectly classified.

A confusion matrix consists of a square grid with rows representing the actual (true) class labels and and columns representing predicted class labels.


In [ ]:
# Generate confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

**CHALLENGE QUESTION:**

1. Is there an imbalance in the class predictions? For example, is one class consistently being wrongly predicted as the other? Or is it fairly balanced?

**Double click here to enter your answers to the questions in this text box.**

<font color="green" size=5>
Answer:
<br>
</font>

#### Support Vector Plot

Let's generate a Support Vector Plot to examine where the support vectors fall in the overall data distribution.

As a reminder, the support vectors are the data points that are closest to the decision boundary or hyperplane. They are the most critical points that determine where the decision boundary lies. NOTE: on this plot, the hyperplane is not visible.

In [ ]:
# Get support vectors
support_vectors = svm_model.support_vectors_

# Plot support vectors
plt.figure(figsize=(8, 8))
plt.scatter(X_train_flat[:, 0], X_train_flat[:, 1], c=y_train, cmap=plt.cm.Paired, marker='o', label='Training Data')
plt.scatter(support_vectors[:, 0], support_vectors[:, 1], s=100, facecolors='none', edgecolors='k', label='Support Vectors')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('Support Vectors')
plt.legend()
plt.show()

Take a look at the support vector plot above. It may be difficult to see all of the data points, so focus on the middle of the graph where there are fewer points to answer the next few questions.

**CHALLENGE QUESTIONS:**

1. Are there a lot of support vectors, or only a few?
2. Imagine trying to draw a line (the hyperplane) that separates all of the support vectors from different classes. Would this be a difficult or easy task?
3. Based on your answer to #2, do you think this was an easy or difficult classification task for the SVM?

**Double click here to enter your answers to the questions in this text box.**

<font color="green" size=5>
Answers:
<br>
<ol>
<li>

</li>
<br>
<li>

</li>
<br>
<li>

</li>
</ol>
</font>

#### Misclassified Images Analysis

To get a better sense of which images within the dataset are being misclassified, we can return to our radiation type PCA plot from the EDA, and label the points on that plot that correspond to images that the SVM misclassified. This will give us further insight into the relationships between the misclassified images.



In [ ]:
# Flatten the images
X = resized_images.reshape(resized_images.shape[0], -1)

# Perform dimensionality reduction using PCA
pca = PCA(n_components=50)  # Adjust the number of components as needed
X_pca = pca.fit_transform(X)

# Perform dimensionality reduction using t-SNE
tsne = TSNE(n_components=2, random_state=42)
X_tsne = tsne.fit_transform(X)

# Obtain the original indices of the test set
_, X_test_indices = train_test_split(range(len(resized_images)), test_size=0.2, random_state=42)

# Identify misclassified points
misclassified_indices = np.where(y_pred != y_test)[0]

# Extract the original indices of the misclassified test samples
misclassified_tsne_indices = [X_test_indices[i] for i in misclassified_indices]

# Plot clusters color-coded by different labels
plt.figure(figsize=(7,7))

# Plot and color by particle type
for label in meta['particle_type'].unique():
    indices = meta[meta['particle_type'] == label].index  # capture indices of images belonging to each class
    plt.scatter(X_tsne[indices, 0], X_tsne[indices, 1], label=label, alpha=0.3)  # match each index to its corresponding row in the reduced dim TSNE

# Highlight misclassified points
plt.scatter(X_tsne[misclassified_tsne_indices, 0], X_tsne[misclassified_tsne_indices, 1], c='red', marker='x', label='Misclassified', alpha=1)

plt.title('Particle Type with Misclassified Points Highlighted')
plt.legend()

plt.show()



Take a look at the PCA plot above, with the SVM misclassified images labeled in red "x", to answer the following questions.

**CHALLENGE QUESTIONS:**

1. Are the misclassified images evenly distributed throughout the PCA plot, or are they clustering in one specific area?
2. What does your answer to #2 tell you about which types of images were most difficult for the SVM to correctly classify? How do you interpret this biologically?


**Double click here to enter your answers to the questions in this text box.**

<font color="green" size=5>
Answers:
<br>
<ol>
<li>

</li>
<br>
<li>

</li>
</ol>
</font>

----

Let's take a look at 10 random misclassified images from the SVM.

In [ ]:
# Select 10 random misclassified indices
random_misclassified_tsne_indices = np.random.choice(misclassified_tsne_indices, 10, replace=False)

# Create a figure for the grid
fig, axes = plt.subplots(2, 5, figsize=(10, 5))
axes = axes.flatten()

# Plot each misclassified image
for ax, idx in zip(axes, random_misclassified_tsne_indices):
  img = load_image_from_s3(idx, meta)
  ax.imshow(img, vmin=400, vmax=4000)
  ax.axis('off')
  true_label = particle_labels['particle_type_encoded'][idx]
  true_display = 'Fe' if true_label == 0 else 'X-ray'
  predicted_label = 0 if true_label == 1 else 1
  predicted_display = 'Fe' if predicted_label == 0 else 'X-ray'
  ax.set_title(f'True: {true_display}, Pred: {predicted_display}')

plt.tight_layout()
plt.show()


Take a look at the misclassified images above. You can re-run the code cell above a few times to get a sense of the patterns.

**CHALLENGE QUESTIONS:**

1. What trends do you observe in the Fe-exposed images that are being misclassified as X-ray?

2. What trends do you observe in the Xray-exposed images that are being misclassified as Fe?

**Double click here to enter your answers to the questions in this text box.**

<font color="green" size=5>
Answers:
<br>
<ol>
<li>

</li>
<br>
<li>

</li>
</ol>
</font>

----

## CNN

Next, let's train the CNN.

> **NOTE:** The cell below takes at least 30 minutes to run because it trains the CNN for 10 epochs. Time for a coffee break! Just make sure that your computer stays awake if you step away.

**Steps that we use for training the CNN below:**

- *Split the dataset into 80% training and 20% validation data.* We refer to the held-out data as validation in this case because it is often used for validating the hyperparameters and architecture of the model. A third step that we will not perform here is to hold-out an additional 20% as a testing set for the final accuracy calculation.
- *Define and use data generators for the training and validation datasets.* This function pulls data from the S3 bucket, resizes each image, flattens each array, and organizes the data and labels in a format that the CNN can read.
- *Define model parameters.*
  - We use **batch size = 32** since this will give us approximately 60 steps per epoch. If the batch size is much smaller than this, an epoch will take a long time, and if it is much larger, the model will lose detail. (Recall that in each epoch, the model iterates through the training data once, and each step in an epoch pulls in data the size of a batch).
  - We use **image input shape** = 140x140 pixels as determined in our EDA. The third value represents the channel, and "1" indicates that each pixel in our images is represented by a single intensity value.
  - We set **number of classes = 2** since we have 2 classes (Fe and X-ray), and we set **number of epochs = 10** so that our model does not train for too long.
  - Finally, **steps per training epoch** and **steps per validation epoch** are simply calculations relying on the number of images in the training and validation sets, divided by the batch size.

- *Build the model.* We are using a simple CNN architecture as outlined below.

  1. **First Convolutional Layer:** This layer looks at small 3x3 patches of the input image and creates 32 different versions of these patches, each emphasizing different features (like edges or textures). It then applies the ReLU function, which helps the model to understand more complex patterns by turning all negative values to zero.
    - In simpler terms, ReLU "turns off" negative values and lets positive values pass through unchanged. This helps the neural network activate (or "fire") only when it detects important features, making the learning process more efficient and effective.

  2. **First Pooling Layer:** This layer reduces the size of the image by half. It does this by picking the highest value from each 2x2 block in the previous layer, which helps to focus on the most important features and makes the computation easier.

  3. **Second Convolutional Layer:** Similar to the first one, but now it uses 64 filters to capture even more detailed features from the already processed image.

  4. **Second Pooling Layer:** Again, this reduces the size by half using the same method as the first pooling layer, further summarizing the features detected.

  5. **Third Convolutional Layer:** This layer uses 128 filters, allowing the model to capture even finer details of the image.

  6. **Third Pooling Layer:** This layer performs another size reduction, focusing on the most significant features detected so far.

  7. **Flatten Layer:** This layer takes the pooled feature maps (which are still in a 2D grid form) and flattens them into a 1D vector. This transformation allows the next layer to process the data.

  8. **First Dense (Fully Connected) Layer:** This layer has 128 neurons and processes the feature map vector, learning complex patterns. It also uses the ReLU function to help the model understand complex patterns by setting all negative numbers to zero.

  9. **Output Layer:** This layer has 2 neurons - 1 for each class. It uses the softmax function to turn the numbers into probabilities that add up to 100%, indicating the likelihood that the image belongs to each class. (For example, an image could be given a 30% likelihood that it belongs to the Fe class, and a 70% likelihood that it belongs to the X-ray class).


In [ ]:
# Import libraries for CNN training and downstream analysis
import os
import boto3
from PIL import Image
from io import BytesIO
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import tensorflow as tf
from botocore import UNSIGNED
from botocore.client import Config

import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import random

In [ ]:
# Split the dataset 80-20 training - validation
train_labels = particle_labels.sample(int(0.8*len(particle_labels))) # get random 80% images and their labels
train_paths = ['Microscopy/train/' + x for x  in list(train_labels['filename'])] # get the filepaths for those images in the same order

val_labels = particle_labels[~particle_labels.isin(train_labels)].dropna() # get the other 20% of the images and their labels
val_paths = ['Microscopy/train/' + x for x  in list(val_labels['filename'])] # and get their filepaths

# Define model parameters
batch_size = 32
input_shape = (140, 140, 1)
num_classes = 2
epochs = 10
steps_per_epoch = len(train_paths) // batch_size
validation_steps = len(val_paths) // batch_size

# Define data generators for training and validation
def generate_batches_from_s3(object_keys, labels_df, batch_size, input_shape):
    indexes = np.arange(len(labels_df))
    while True:
        for i in range(0, len(labels_df), batch_size):
            num_rows = len(labels_df)
            batch_indexes = indexes[i:i+batch_size]
            X = np.empty((len(batch_indexes), *input_shape))
            y = np.empty((len(batch_indexes)), dtype=int)
            for j, idx in enumerate(batch_indexes):
                row = labels_df.iloc[idx]
                img_filename = row['filename']
                img_label = 0 if row['particle_type'] == 'Fe' else 1
                obj = s3_client.get_object(Bucket=s3_bucket, Key=os.path.join(s3_dir_path, img_filename))
                img_bytes = obj['Body'].read()
                img = Image.open(BytesIO(img_bytes))
                img = img.resize((input_shape[0], input_shape[1]))
                img = np.array(img)
                img = np.clip(img, 400, 4000) # mimicking imshow vmin/vmax https://stackoverflow.com/questions/31232733/vmin-vmax-algorithm-matplotlib
                X[j] = np.expand_dims(img, axis=-1)
                y[j] = img_label
            yield X, y


# Create data generators for training and validation
train_dataset = tf.data.Dataset.from_generator(
    lambda: generate_batches_from_s3(train_paths, train_labels, batch_size, input_shape),
    output_types=(tf.float32, tf.int32),
    output_shapes=(tf.TensorShape([None, *input_shape]), tf.TensorShape([None])),
).repeat()

validation_dataset = tf.data.Dataset.from_generator(
    lambda: generate_batches_from_s3(val_paths, val_labels, batch_size, input_shape),
    output_types=(tf.float32, tf.int32),
    output_shapes=(tf.TensorShape([None, *input_shape]), tf.TensorShape([None])),
).repeat()

# Build CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model_history = model.fit(train_dataset, epochs=epochs, steps_per_epoch=steps_per_epoch, validation_data=validation_dataset, validation_steps=validation_steps)


**CHALLENGE QUESTIONS:**

1. What was the final training accuracy?

2. What was the final validation accuracy?

3. Based on accuracy alone, which model performed better at our classification task, SVM or CNN?

**Double click here to enter your answers to the questions in this text box.**

<font color="green" size=5>
Answers:
<br>
<ol>
<li>

</li>
<br>
<li>

</li>
<br>
<li>

</li>
</ol>
</font>

#### Model Accuracy Plot

Let's create a model accuracy plot to see how the training and validation accuracy changed over time.

In [ ]:
import matplotlib.pyplot as plt

train_accuracy = model_history.history['accuracy']
val_accuracy = model_history.history['val_accuracy']
epochs = range(1, len(train_accuracy) + 1)

plt.plot(epochs, train_accuracy, 'b', label='Training Accuracy')
plt.plot(epochs, val_accuracy, 'r', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()


**CHALLENGE QUESTIONS:**

1. Using the accuracy plot above, do you see any signs of overfitting? What about underfitting?

2. Do you think that we should train the model for more epochs, or do you think that we have reached the best possible accuracy for this model and this subset of the dataset?

**Double click here to enter your answers to the questions in this text box.**

<font color="green" size=5>
Answers:
<br>
<ol>
<li>

</li>
<br>
<li>

</li>
</ol>
</font>

#### Misclassified Images Analysis

Just like we did after training the SVM, we can return to our radiation type PCA plot from the EDA, and label the points on that plot that correspond to images that the SVM misclassified. This will give us further insight into the relationships between the misclassified images.

**NOTE: the cell below takes a few minutes to run.**

In [ ]:
# Flatten the images
X = resized_images.reshape(resized_images.shape[0], -1)

# Perform dimensionality reduction using PCA
pca = PCA(n_components=50)
X_pca = pca.fit_transform(X)

# Perform dimensionality reduction using t-SNE
tsne = TSNE(n_components=2, random_state=42)
X_tsne = tsne.fit_transform(X)

# Initialize arrays to store prediction results
predicted_labels = []
true_labels = []

# Iterate over the validation dataset and collect predictions
for i, (X_batch, y_batch) in enumerate(validation_dataset.take(validation_steps)):
    batch_predictions = model.predict(X_batch)
    batch_predicted_labels = np.argmax(batch_predictions, axis=1)
    predicted_labels.extend(batch_predicted_labels)
    true_labels.extend(y_batch.numpy())

    #print(f"Step {i+1}/{validation_steps} completed.")

# Convert lists to numpy arrays
predicted_labels = np.array(predicted_labels)
true_labels = np.array(true_labels)

# Identify misclassified indices
misclassified_indices = np.where(predicted_labels != true_labels)[0]


# Print the number of misclassified images
num_misclassified = len(misclassified_indices)
print(f"Number of misclassified images: {num_misclassified}")


# Plot clusters color-coded by different labels
plt.figure(figsize=(7,7))

# Plot and color by particle type

for label in meta['particle_type'].unique():
    indices = meta[meta['particle_type'] == label].index
    plt.scatter(X_tsne[indices, 0], X_tsne[indices, 1], label=label, alpha=0.5)
# Highlight misclassified points
plt.scatter(X_tsne[misclassified_indices, 0], X_tsne[misclassified_indices, 1], color='red', marker='x', label='Misclassified', alpha=0.7)
plt.title('Particle Type')
plt.legend()

plt.show()


Take a look at the PCA plot above, with the CNN misclassified images labeled in red "x", to answer the following questions.

**CHALLENGE QUESTIONS:**

1. Are the misclassified images evenly distributed throughout the PCA plot, or are they clustering in one specific area?
2. What does your answer to #2 tell you about which types of images were most difficult for the CNN to correctly classify? How do you interpret this biologically?

**Double click here to enter your answers to the questions in this text box.**

<font color="green" size=5>
Answers:
<br>
<ol>
<li>

</li>
<br>
<li>

</li>
</ol>
</font>

----

Let's take a look at 10 random misclassified images *and* 10 random correctly classified images from the CNN.

**10 Random Misclassified Images from CNN:**

In [ ]:
# Select 10 random misclassified indices
random_misclassified_indices = np.random.choice(misclassified_indices, 10, replace=False)

# Create a figure for the grid
fig, axes = plt.subplots(2, 5, figsize=(10, 5))
axes = axes.flatten()

# Plot each misclassified image
for ax, idx in zip(axes, random_misclassified_indices):
  img = load_image_from_s3(idx, val_labels)
  ax.imshow(img, vmin=400, vmax=4000)
  ax.axis('off')
  true_label = true_labels[idx]
  true_display = 'Fe' if true_label == 0 else 'X-ray'
  predicted_label = predicted_labels[idx]
  predicted_display = 'Fe' if predicted_label == 0 else 'X-ray'
  ax.set_title(f'True: {true_label}, Pred: {predicted_label}')

plt.tight_layout()
plt.show()


----

**10 Random Correctly Classified Images from CNN:**

In [ ]:
correct_indices = np.where(predicted_labels == true_labels)[0]

# Select 10 random correctly classified indices
random_correct_indices = np.random.choice(correct_indices, 10, replace=False)

# Create a figure for the grid
fig, axes = plt.subplots(2, 5, figsize=(10, 5))
axes = axes.flatten()

# Plot each correctly classified image
for ax, idx in zip(axes, random_correct_indices):
  img = load_image_from_s3(idx, val_labels)
  ax.imshow(img, vmin=400, vmax=4000)
  ax.axis('off')
  true_label = true_labels[idx]
  predicted_label = predicted_labels[idx]
  ax.set_title(f'True: {true_label}, Pred: {predicted_label}')

plt.tight_layout()
plt.show()


----

You can rerun the two cells above to get a sense of which types of images are being misclassified.  

**CHALLENGE QUESTIONS:**

1. What patterns do you see consistently in the Fe images that are being misclassified as X-ray? What about the Fe images that are being *correctly* classified as X-ray?

2. What patterns do you see consistently in the X-ray images that are being misclassified as Fe? What about the X-ray images that are being *correctly* classified as X-ray?



**Double click here to enter your answers to the questions in this text box.**

<font color="green" size=5>
Answers:
<br>
<ol>
<li>

</li>
<br>
<li>

</li>
</ol>
</font>

# Sex Classifier

Now that we've successfully trained and evaluated two different classifiers to predict the *radiation exposure type* for each image, let's return to the other questions we outlined at the beginning of this project:

- Can we train a machine learning classifier to classify the *sex* of the mouse from which each cell nucleus 53bp1+ DNA damage image was taken?
  - What about the *strain* of the mouse?
- If not, what does this tell us about what biological traits are being captured in the 53bp1+ DNA damage cell nucleus images?

----

Let's train the same SVM and CNN architectures but instead we will use the *sex* of the mouse that each cellular image came from as the labels.

In [ ]:
# Get the sex labels only
sex_labels = meta[['filename','sex']]
sex_labels

## SVM

We will train an SVM on the sex labels below.

In [ ]:
# Import libraries for SVM and downstream analysis
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import seaborn as sns
from sklearn.metrics import confusion_matrix

import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import random

In [ ]:
# Encoding labels
label_encoder = LabelEncoder()
sex_labels['sex_type_encoded'] = label_encoder.fit_transform(sex_labels['sex'])

# Load and preprocess images
X = resized_images
y = sex_labels['sex_type_encoded'].values

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Flatten image arrays
X_train_flat = X_train.reshape(X_train.shape[0], -1)
X_test_flat = X_test.reshape(X_test.shape[0], -1)

# Train SVM model
svm_model = SVC(kernel='rbf')
svm_model.fit(X_train_flat, y_train)

# Predictions
y_pred = svm_model.predict(X_test_flat)

# Evaluate model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Predictions
y_pred = svm_model.predict(X_test_flat)

# Generate confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()


**CHALLENGE QUESTION:**

1. How did the SVM trained on the sex of the mice compare to the SVM trained on the radiation exposure type?

**Double click here to enter your answers to the questions in this text box.**

<font color="green" size=5>
Answer:
<br>
</font>

## CNN

In [ ]:
# Import libraries for CNN training and downstream analysis
import os
import boto3
from PIL import Image
from io import BytesIO
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import tensorflow as tf
from botocore import UNSIGNED
from botocore.client import Config

import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import random

In [ ]:
# Split the dataset 80-20 training - validation
train_labels = sex_labels.sample(int(0.8*len(sex_labels))) # get random 80% images and their labels
train_paths = ['Microscopy/train/' + x for x  in list(train_labels['filename'])] # get the filepaths for those images in the same order

val_labels = sex_labels[~sex_labels.isin(train_labels)].dropna() # get the other 20% of the images and their labels
val_paths = ['Microscopy/train/' + x for x  in list(val_labels['filename'])] # and get their filepaths

# Define model parameters
batch_size = 32
input_shape = (140, 140, 1)
num_classes = 2
epochs = 10
steps_per_epoch = len(train_paths) // batch_size
validation_steps = len(val_paths) // batch_size

# Define data generators for training and validation
def generate_batches_from_s3(object_keys, labels_df, batch_size, input_shape):
    indexes = np.arange(len(labels_df))
    while True:
        for i in range(0, len(labels_df), batch_size):
            num_rows = len(labels_df)
            batch_indexes = indexes[i:i+batch_size]
            X = np.empty((len(batch_indexes), *input_shape))
            y = np.empty((len(batch_indexes)), dtype=int)
            for j, idx in enumerate(batch_indexes):
                row = labels_df.iloc[idx]
                img_filename = row['filename']
                img_label = 0 if row['sex'] == 'FEMALE' else 1
                obj = s3_client.get_object(Bucket=s3_bucket, Key=os.path.join(s3_dir_path, img_filename))
                img_bytes = obj['Body'].read()
                img = Image.open(BytesIO(img_bytes))
                img = img.resize((input_shape[0], input_shape[1]))
                img = np.array(img)
                img = np.clip(img, 400, 4000) # mimicking imshow vmin/vmax https://stackoverflow.com/questions/31232733/vmin-vmax-algorithm-matplotlib
                X[j] = np.expand_dims(img, axis=-1)
                y[j] = img_label
            yield X, y


# Create data generators for training and validation
train_dataset = tf.data.Dataset.from_generator(
    lambda: generate_batches_from_s3(train_paths, train_labels, batch_size, input_shape),
    output_types=(tf.float32, tf.int32),
    output_shapes=(tf.TensorShape([None, *input_shape]), tf.TensorShape([None])),
).repeat()

validation_dataset = tf.data.Dataset.from_generator(
    lambda: generate_batches_from_s3(val_paths, val_labels, batch_size, input_shape),
    output_types=(tf.float32, tf.int32),
    output_shapes=(tf.TensorShape([None, *input_shape]), tf.TensorShape([None])),
).repeat()

# Build CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model_history = model.fit(train_dataset, epochs=epochs, steps_per_epoch=steps_per_epoch, validation_data=validation_dataset, validation_steps=validation_steps)


In [ ]:
import matplotlib.pyplot as plt

# Assuming `history` is the History object returned by model.fit()
train_accuracy = model_history.history['accuracy']
val_accuracy = model_history.history['val_accuracy']
epochs = range(1, len(train_accuracy) + 1)

plt.plot(epochs, train_accuracy, 'b', label='Training Accuracy')
plt.plot(epochs, val_accuracy, 'r', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()


**CHALLENGE QUESTIONS:**

1. How did the CNN trained on the sex of the mice perform compared to the CNN trained on the radiation exposure type?

2. How did the CNN trained on the sex of the mice perform compared to the SVM trained on the sex of the mice?

3. Based on these results, do you think the sex of the mice is being captured in the 53bp1+ DNA damage patterns of the images?

**Double click here to enter your answers to the questions in this text box.**

<font color="green" size=5>
Answers:
<br>
<ol>
<li>

</li>
<br>
<li>

</li>
<br>
<li>

</li>
</ol>
</font>

# Strain Classifier

Let's return again to the questions at the beginning of this project:

- Can we train a machine learning classifier to classify the *sex* of the mouse from which each cell nucleus 53bp1+ DNA damage image was taken?
  - What about the *strain* of the mouse?
- If not, what does this tell us about what biological traits are being captured in the radiation images?

----

Let's train the same SVM and CNN architectures but instead we will use the *strain* of the mouse that each cellular image came from as the labels.

In [ ]:
# Get the strain labels only
strain_labels = meta[['filename','strain']]
strain_labels

## SVM

We will train an SVM on the strain labels below.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import seaborn as sns
from sklearn.metrics import confusion_matrix


In [ ]:
# Encoding labels
label_encoder = LabelEncoder()
strain_labels['strain_type_encoded'] = label_encoder.fit_transform(strain_labels['strain'])

# Load and preprocess images
X = resized_images
y = strain_labels['strain_type_encoded'].values

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Flatten image arrays
X_train_flat = X_train.reshape(X_train.shape[0], -1)
X_test_flat = X_test.reshape(X_test.shape[0], -1)

# Train SVM model
svm_model = SVC(kernel='rbf')
svm_model.fit(X_train_flat, y_train)

# Predictions
y_pred = svm_model.predict(X_test_flat)

# Evaluate model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Predictions
y_pred = svm_model.predict(X_test_flat)

# Generate confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()


**CHALLENGE QUESTION:**

1. How did the SVM trained on the strain of the mice perform compared to the SVMs trained on the sex of the mice and the radiation exposure type?

**Double click here to enter your answers to the questions in this text box.**

<font color="green" size=5>
Answer:
<br>
</font>

## CNN

In [ ]:
# Import libraries for CNN training and downstream analysis
import os
import boto3
from PIL import Image
from io import BytesIO
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import tensorflow as tf
from botocore import UNSIGNED
from botocore.client import Config

import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import random

In [ ]:
# Split the dataset 80-20 training - validation
train_labels = strain_labels.sample(int(0.8*len(strain_labels))) # get random 80% images and their labels
train_paths = ['Microscopy/train/' + x for x  in list(train_labels['filename'])] # get the filepaths for those images in the same order

val_labels = strain_labels[~strain_labels.isin(train_labels)].dropna() # get the other 20% of the images and their labels
val_paths = ['Microscopy/train/' + x for x  in list(val_labels['filename'])] # and get their filepaths

# Define model parameters
batch_size = 32
input_shape = (140, 140, 1)
num_classes = 2
epochs = 10
steps_per_epoch = len(train_paths) // batch_size
validation_steps = len(val_paths) // batch_size

# Define data generators for training and validation
def generate_batches_from_s3(object_keys, labels_df, batch_size, input_shape):
    indexes = np.arange(len(labels_df))
    while True:
        for i in range(0, len(labels_df), batch_size):
            num_rows = len(labels_df)
            batch_indexes = indexes[i:i+batch_size]
            X = np.empty((len(batch_indexes), *input_shape))
            y = np.empty((len(batch_indexes)), dtype=int)
            for j, idx in enumerate(batch_indexes):
                row = labels_df.iloc[idx]
                img_filename = row['filename']
                img_label = 0 if row['strain'] == 'BALBC' else 1
                obj = s3_client.get_object(Bucket=s3_bucket, Key=os.path.join(s3_dir_path, img_filename))
                img_bytes = obj['Body'].read()
                img = Image.open(BytesIO(img_bytes))
                img = img.resize((input_shape[0], input_shape[1]))
                img = np.array(img)
                img = np.clip(img, 400, 4000) # mimicking imshow vmin/vmax https://stackoverflow.com/questions/31232733/vmin-vmax-algorithm-matplotlib
                X[j] = np.expand_dims(img, axis=-1)
                y[j] = img_label
            yield X, y


# Create data generators for training and validation
train_dataset = tf.data.Dataset.from_generator(
    lambda: generate_batches_from_s3(train_paths, train_labels, batch_size, input_shape),
    output_types=(tf.float32, tf.int32),
    output_shapes=(tf.TensorShape([None, *input_shape]), tf.TensorShape([None])),
).repeat()

validation_dataset = tf.data.Dataset.from_generator(
    lambda: generate_batches_from_s3(val_paths, val_labels, batch_size, input_shape),
    output_types=(tf.float32, tf.int32),
    output_shapes=(tf.TensorShape([None, *input_shape]), tf.TensorShape([None])),
).repeat()

# Build CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model_history = model.fit(train_dataset, epochs=epochs, steps_per_epoch=steps_per_epoch, validation_data=validation_dataset, validation_steps=validation_steps)


In [ ]:
train_accuracy = model_history.history['accuracy']
val_accuracy = model_history.history['val_accuracy']
epochs = range(1, len(train_accuracy) + 1)

plt.plot(epochs, train_accuracy, 'b', label='Training Accuracy')
plt.plot(epochs, val_accuracy, 'r', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()


**CHALLENGE QUESTIONS:**

1. How did the CNN trained on the strain of the mice perform compared to the CNNs trained on the sex of the mice and the radiation exposure type?

2. How did the CNN trained on the strain of the mice perform compared to the SVM trained on the strain of the mice?

3. Based on the performance of each classifier, which aspect(s) of the 53bp1+ DNA damage images do you think the strain of the mice is affecting?

**Double click here to enter your answers to the questions in this text box.**

<font color="green" size=5>
Answers:
<br>
<ol>
<li>

</li>
<br>
<li>

</li>
<br>
<li>

</li>
</ol>
</font>

# Choose Your Own Adventure (Optional)

Now that we have answered the questions we set out at the beginning of this project, there are multiple ways you could take this project further if you are interested in writing some of your own code.

- Transfer learning (start with a pretrained image model)
- Segment DNA damage foci
- xAI - SHAP, LIME, Captum, attention maps
- Regression to predict nfoci

## Transfer Learning

In the sections above, we trained an SVM and a CNN from scratch. However, in some cases, better performance has been shown for image classification when using a pretrained model.

Several official Tensorflow pretrained models for image classification are available through GitHub: https://github.com/tensorflow/models/tree/master/official#image-classification

There are also hundreds of pretrained image classification models on HuggingFace: https://huggingface.co/models?sort=trending&search=image+classification

Feel free to load one of these models and test it out on this dataset to see if you can improve performance!

> Don't forget to evaluate which images are being misclassified and why.

## Segmentation

In the sections above, we relied on each model identify important features from the images, such as the fluorescent foci that indicate DNA damage.

<img src="https://drive.google.com/uc?id=1OS3kJDg88GPwZeFcCEKtr3eDANj4nBw_">

However, an auto-segmentation model would be useful to identify and quantify the number of distinct DNA damage foci per image, in each class.

Once again, several image segmentation models are available on Tensorflow GitHub or on HuggingFace:

https://github.com/tensorflow/models/blob/master/official/vision/README.md#object-detection-and-instance-segmentation

https://huggingface.co/models?sort=trending&search=image+segmentation

You could also train your own image segmentation model: https://www.tensorflow.org/tutorials/images/segmentation

> The end result of this effort should be a quantification of the number of distinct DNA damage fluorescent foci per image.

## Explainable AI (xAI) for CNN

In the sections above, we tried to understand which features the CNN was paying attention to, by identifying the misclassified images and applying our knowledge of biology.

There are several Python libraries which are specifically designed to identify the areas of an image that a CNN is using for classification.

It would be very useful to implement one of these libraries to understand which areas of the image the CNN is focused on. We can then make a biological interpretation.

GradCAM: https://keras.io/examples/vision/grad_cam/

SHAP: https://shap.readthedocs.io/en/latest/image_examples.html

Feel free to explore on your own and identify other useful libraries!

> The end result of this effort should be code that creates a heatmap overlaied on one of the DNA damage microscopy images, where the colors of the heatmap indicate the areas that were used by the CNN for classification.

